In [1]:
import pandas as pd
import json
import os

In [31]:
dir_name = 'C:/Users/KIHyuk/Desktop/brunch_data/json'

def get_file_list(dir_name): # file name들을 가져오는 함수 # 폴더명 인자 # 폴더가 위치한 경로를 인자로
    return os.listdir(dir_name) # 폴더 내 파일명을 리스트 형태로 반환 

file_list = get_file_list(dir_name)

In [32]:
def pre_keyword(x):
    tmp = []
    for val in x:
        tmp.append(val.replace("\n","").replace(" ",""))
    return tmp

def pre_comment(x):
    if len(x) == 0:
        return None
    else :
        return x

def pre_text(x):
    return str(x)

In [33]:
all_df = pd.DataFrame(columns=['class','text'])
each_df = {}

for file in file_list:
    with open('C:/Users/KIHyuk/Desktop/brunch_data/json/'+file,encoding='UTF8') as json_file:
        json_data = json.load(json_file)
    
    df = pd.DataFrame(json_data['data'],
                  columns=['title','keyword','text','nickname','publish_date','likes','share','comment','url','url_plink'])
    df = df.dropna(subset=['text'])
    df['keyword'] = df['keyword'].apply(pre_keyword)
    df['comment'] = df['comment'].apply(pre_comment)
    df['text'] = df['text'].apply(pre_text)
    df.insert(0,"class",file[:-5])
    
    all_df = pd.concat([all_df,df[['class','text']][:500]])
    each_df[file[:-5]] = df

In [44]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(all_df['text'],all_df['class'],test_size=0.2 ,shuffle=True)

In [48]:
from konlpy.tag import Twitter

twitter = Twitter()

def tw_tokenizer(text):
    tokens_ko = twitter.morphs(text)
    return tokens_ko

C:\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Anaconda3\lib\site-packages\jpype\_core.py:217: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(X_train)
tfidf_matrix_train = tfidf_vect.transform(X_train)

C:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
lg_clf = LogisticRegression(random_state=0)

params = {'C' :[1, 3.5, 4.5, 5.5, 10]}

grid_cv = GridSearchCV(lg_clf, parma_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train,y_train)
print(grid_cv.best_params_, round(grid_cv.best_score_,4))